In [1]:
import tensorflow as tf


import pandas as pd
from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer



import the upsampled file from PART II

In [2]:
upsampled = pd.read_csv('/upsampled.csv')
upsampled['speech'] = upsampled['speech'].astype('str') 
upsampled.shape

(450000, 8)

In [3]:
upsampled.dtypes

Unnamed: 0          int64
member_name        object
sitting_date        int64
political_party    object
speaker_info       object
speech             object
speech_length       int64
binned             object
dtype: object

convert political party to **integer**

In [4]:
upsampled['political_party'] = pd.Categorical(upsampled['political_party'])
upsampled['political_party'] = upsampled.political_party.cat.codes

In [5]:
upsampled['political_party'] 

0          6
1         13
2          7
3          4
4          3
          ..
449995     6
449996    14
449997    14
449998     1
449999    12
Name: political_party, Length: 450000, dtype: int8

In [6]:
upsampled = upsampled[['speech','political_party']]
upsampled

,speech,political_party
0,φυσικά οφείλομαι πρόβλημα αμέλεια υπηρεσία λέω...,6
1,ευχαριστώ και σκυλλάκο επαναλάβω λέω χθες ομιλ...,13
2,αρχάς θέλω ευχαριστήσω συναδέλφου έδωσαν προτε...,7
3,νόμος τελείωσε πράγματο διευκρινίσω γίνω γίνω ...,4
4,δωδέκατη αριθμό επίκαιρη ερώτηση δεύτερου κύκλ...,3
...,...,...
449995,λέω φτάσαμε ελλάδα έφτασε χρεοκοπίας τρίγωνο λ...,6
449996,μπορώ ζητήσω διευκρίνιση νομοτεχνικός βελτίωση...,14
449997,προσέξτε συμβώ λέγαμε στοιχείο εκδότα διαχειρι...,14
449998,μπορώ λεπτό θέλω ευχέρεια προσθέσω,1


Get 20% of data for testing (90000 speeches)

In [7]:

upsampled = upsampled.sample(frac=1)
upsampled.reset_index(drop=True,inplace=True)
test_size = int(len(upsampled) * 0.2) # the test data will be 20% (20) of the entire data

# the copy() here is important, it will prevent us from getting: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame. Try using .loc[row_index,col_indexer] = value instead
test = upsampled.iloc[-test_size:,:].copy()
print( test.shape)

(90000, 2)


In [8]:
upsampled.iloc[:-test_size,:]

,speech,political_party
0,πεπονή συγχωρείτε κεφάλαιο δεύτερο ό τίτλο νομ...,13
1,θέλω συζητηθώ άρθρο συζητηθώ συναφός,9
2,δω ανάλογη ευαισθησία επιθέσει βάρος μελής κόμ...,8
3,παρακαλώ μιλήσω τσιπλάκο δέχομαι διακοπή κρατε...,11
4,στουρνάρας ελληνικός λαός υποστώ θυσία πιάνω τ...,8
...,...,...
359995,λέμε βεβαίως εκπρόθεσμη τροπολογίο λέμε εκπρόθ...,14
359996,διαφωνώ λέτε υφυπουργέ κάνω έκκληση απαλειφθώ ...,11
359997,ευχαριστούμε βουλευτής νίκη κεραμέω ζητώ άδεια...,12
359998,σύμφωνα ασφαλείς διπλωματικός πληροφορία γενικ...,8


Split the rest into Train (306000) and Validation (54000) data

In [9]:
train_val = upsampled.iloc[:-test_size,:].copy() 
val_size = int(len(train_val) * 0.15) 
val = train_val.iloc[-val_size:,:].copy()
train = train_val.iloc[:-val_size,:].copy()
print( train.shape,val.shape)

(306000, 2) (54000, 2)


In [10]:
train

,speech,political_party
0,πεπονή συγχωρείτε κεφάλαιο δεύτερο ό τίτλο νομ...,13
1,θέλω συζητηθώ άρθρο συζητηθώ συναφός,9
2,δω ανάλογη ευαισθησία επιθέσει βάρος μελής κόμ...,8
3,παρακαλώ μιλήσω τσιπλάκο δέχομαι διακοπή κρατε...,11
4,στουρνάρας ελληνικός λαός υποστώ θυσία πιάνω τ...,8
...,...,...
305995,ευχαριστώ ξεκινήσω περιγραφή κατάσταση βλέπω χ...,14
305996,τσοβόλα επικαλείται κανονισμός ζητώ προεδρείο ...,13
305997,νομίζω κάνω ελέγχω μηχανισμού κράτου και ελέγχ...,3
305998,ευχαριστώ εκπλήσσω προσωπικά φορά όριο απόψει ...,12


In [11]:
val

,speech,political_party
306000,σημαντικά δυνατότητα δω υπουργό βλέπω,1
306001,ευχαριστώ υφυπουργέ θέλω δικός άποψη λέω προηγ...,7
306002,ευχαριστώ ξεκινήσω νέος ευχάριστος ξεκινάω ευχ...,13
306003,παρακαλώ τηρείται χρόνος υπάρχω περιθώριο πολυ...,12
306004,παρακαλώ συνάδελφε εοκ νατος συνδικάτο λέγατε ...,13
...,...,...
359995,λέμε βεβαίως εκπρόθεσμη τροπολογίο λέμε εκπρόθ...,14
359996,διαφωνώ λέτε υφυπουργέ κάνω έκκληση απαλειφθώ ...,11
359997,ευχαριστούμε βουλευτής νίκη κεραμέω ζητώ άδεια...,12
359998,σύμφωνα ασφαλείς διπλωματικός πληροφορία γενικ...,8


In [12]:
test

,speech,political_party
360000,κάνατε αναφορά υποθέσεω γνωρίζω πράγματι μεγάλ...,2
360001,μίλησα προσωπικώς βάρος ακούγατε είπα δίνατε α...,4
360002,σαμαράς αναφέρθηκα βάλτε δικλίδα ασφαλεία μετα...,7
360003,βορίδη ευχαριστώ αναφερθήκατε ξεκαθαρίσω πείρα...,12
360004,παρακαλώ διάλογος κυρία τιμή ανακοινώσω σώμα δ...,12
...,...,...
449995,τιμή ανακοινώσω συνάδελφος βουλευτής αθηνών μα...,10
449996,τελειώνω θέλω ευχαριστώ ανοχή υπάρχω προβλήματ...,2
449997,ευχαριστώ κύριος υπουργός χρυσανθακόπουλος αιθ...,12
449998,κοιτάξτε καθαρά βουλή απόστολος κακλαμάνης πρό...,3


In [13]:
train.speech

0         πεπονή συγχωρείτε κεφάλαιο δεύτερο ό τίτλο νομ...
1                      θέλω συζητηθώ άρθρο συζητηθώ συναφός
2         δω ανάλογη ευαισθησία επιθέσει βάρος μελής κόμ...
3         παρακαλώ μιλήσω τσιπλάκο δέχομαι διακοπή κρατε...
4         στουρνάρας ελληνικός λαός υποστώ θυσία πιάνω τ...
                                ...                        
305995    ευχαριστώ ξεκινήσω περιγραφή κατάσταση βλέπω χ...
305996    τσοβόλα επικαλείται κανονισμός ζητώ προεδρείο ...
305997    νομίζω κάνω ελέγχω μηχανισμού κράτου και ελέγχ...
305998    ευχαριστώ εκπλήσσω προσωπικά φορά όριο απόψει ...
305999                παρακαλώ ανέβω συνάδελφό βήμα κοντονή
Name: speech, Length: 306000, dtype: object

count the frequency of each word and the total number of words in the corpus

In [14]:
from collections import Counter 

#count unique words
def word_counter(text):
  count = Counter()
  for i in text.values:
     for word in i.split():
       count[word]+=1
  return count


In [15]:
corpus = upsampled.speech
counter = word_counter(corpus)
len(counter)

32539

In [16]:
counter

Counter({'πεπονή': 527,
         'συγχωρείτε': 4418,
         'κεφάλαιο': 13653,
         'δεύτερο': 21828,
         'ό': 54504,
         'τίτλο': 3470,
         'νομοσχεδίο': 29769,
         'διατάξει': 30029,
         'θέλω': 279695,
         'συζητηθώ': 22370,
         'άρθρο': 66300,
         'συναφός': 1266,
         'δω': 20555,
         'ανάλογη': 2085,
         'ευαισθησία': 9687,
         'επιθέσει': 1522,
         'βάρος': 18532,
         'μελής': 5434,
         'κόμματος': 19323,
         'δέχομαι': 12951,
         'γραφείο': 13790,
         'επανέλθω': 5967,
         'επικοινωνία': 5723,
         'φοιτητής': 6597,
         'δήλωσαν': 453,
         'στοιχείο': 56243,
         'κουκουλοφόρου': 370,
         'διάφορους': 2727,
         'λυμαίνομαι': 350,
         'χώρου': 6666,
         'προβαίνω': 1456,
         'καταγγελία': 7927,
         'ζητείται': 1188,
         'συνεχής': 1296,
         'επέμβαση': 1742,
         'αστυνομία': 11820,
         'πανεπιστημίο': 4996,
      

define max lenght of each speech to 150

In [17]:
num_words = len(counter)
max_len = 150

split each dataframe(train,val,test) to speech and label (target)

In [18]:
train_speeches = train.speech
train_labels = train.political_party

val_speeches = val.speech
val_labels = val.political_party

test_speeches = test.speech
test_labels = test.political_party

tokenize the words of training data

In [19]:
tok = Tokenizer(num_words=num_words)
tok.fit_on_texts(train_speeches)

print the list of words with their unique code 

In [20]:
word_index = tok.word_index
word_index

{'λέω': 1,
 'κάνω': 2,
 'θέλω': 3,
 'χώρα': 4,
 'υπουργός': 5,
 'βουλή': 6,
 'ξέρω': 7,
 'ελληνικός': 8,
 'ευχαριστώ': 9,
 'δημοκρατία': 10,
 'υπουργείο': 11,
 'κυρία': 12,
 'πολιτικός': 13,
 'νέος': 14,
 'πηγαίνω': 15,
 'υπάρχω': 16,
 'χρόνια': 17,
 'επιτροπή': 18,
 'βουλευτής': 19,
 'στιγμή': 20,
 'ζήτημα': 21,
 'διαδικασία': 22,
 'πάρω': 23,
 'συζήτηση': 24,
 'κύριος': 25,
 'υπουργό': 26,
 'νόμο': 27,
 'τροπολογίο': 28,
 'φορά': 29,
 'ελλάδα': 30,
 'ερώτηση': 31,
 'νομίζω': 32,
 'πολιτική': 33,
 'βλέπω': 34,
 'τρόπο': 35,
 'ευρώ': 36,
 'εθνικός': 37,
 'χρόνο': 38,
 'και': 39,
 'πρόβλημα': 40,
 'έργο': 41,
 'δώσω': 42,
 'άρθρο': 43,
 'δημόσιος': 44,
 'μπορώ': 45,
 'μεγάλος': 46,
 'μιλώ': 47,
 'οικονομικός': 48,
 'ευρωπαϊκός': 49,
 'κοινωνικός': 50,
 'λαός': 51,
 'σώμα': 52,
 'αριθμό': 53,
 'στοιχείο': 54,
 'αρχή': 55,
 'έρχομαι': 56,
 'μέτρο': 57,
 'προβλήματο': 58,
 'γνωρίζω': 59,
 'ανάπτυξη': 60,
 'ό': 61,
 'υγεία': 62,
 'υπηρεσία': 63,
 'έπρεπε': 64,
 'νέας': 65,
 'λειτουργία': 66

Get the sequences of training data 

In [21]:

train_sequences = tok.texts_to_sequences(train_speeches)

In [22]:
train_sequences[2]

[281,
 2888,
 694,
 3649,
 319,
 1240,
 299,
 511,
 3649,
 467,
 1153,
 1180,
 1036,
 8728,
 54,
 9714,
 2313,
 10451,
 1017,
 3868,
 840,
 4381,
 4313,
 3329,
 557,
 1358,
 172,
 1036,
 1258,
 3381,
 3886,
 1688,
 1688,
 13449,
 3372,
 399,
 3,
 7120,
 249,
 12569,
 3,
 2,
 3070,
 197,
 88,
 1046,
 82,
 3505,
 4080,
 1688,
 3505,
 34,
 7806,
 3329,
 6800,
 1359,
 631,
 5563,
 4198,
 34,
 2107,
 12236,
 149,
 13258,
 464,
 1,
 9714,
 1706,
 17,
 212,
 7087,
 7087,
 9410,
 8494,
 3649,
 839,
 425,
 1036,
 228,
 5542,
 5583,
 1299,
 14417,
 2732,
 7168,
 359,
 16623,
 399,
 74,
 3254,
 181,
 157,
 499,
 16190,
 13258,
 464,
 4393,
 7807,
 969,
 126,
 411,
 192,
 547,
 851,
 1706,
 172,
 4730,
 306,
 1558,
 1414,
 5720,
 7448,
 19773,
 58,
 2443,
 11879,
 349,
 44,
 62,
 44,
 512,
 32,
 44,
 1318,
 2150,
 58,
 97,
 1276,
 958,
 97,
 73]

pad each speech 

In [23]:
from keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(train_sequences,padding='post',truncating='post',maxlen=max_len)

In [24]:
train_padded[2]

array([  281,  2888,   694,  3649,   319,  1240,   299,   511,  3649,
         467,  1153,  1180,  1036,  8728,    54,  9714,  2313, 10451,
        1017,  3868,   840,  4381,  4313,  3329,   557,  1358,   172,
        1036,  1258,  3381,  3886,  1688,  1688, 13449,  3372,   399,
           3,  7120,   249, 12569,     3,     2,  3070,   197,    88,
        1046,    82,  3505,  4080,  1688,  3505,    34,  7806,  3329,
        6800,  1359,   631,  5563,  4198,    34,  2107, 12236,   149,
       13258,   464,     1,  9714,  1706,    17,   212,  7087,  7087,
        9410,  8494,  3649,   839,   425,  1036,   228,  5542,  5583,
        1299, 14417,  2732,  7168,   359, 16623,   399,    74,  3254,
         181,   157,   499, 16190, 13258,   464,  4393,  7807,   969,
         126,   411,   192,   547,   851,  1706,   172,  4730,   306,
        1558,  1414,  5720,  7448, 19773,    58,  2443, 11879,   349,
          44,    62,    44,   512,    32,    44,  1318,  2150,    58,
          97,  1276,

get the sequences for test and validation data, also apply the padding at the end 

In [25]:
val_sequences = tok.texts_to_sequences(val_speeches)
test_sequences = tok.texts_to_sequences(test_speeches)

val_padded = pad_sequences(val_sequences,padding='post',truncating='post',maxlen=max_len)
test_padded = pad_sequences(test_sequences,padding='post',truncating='post',maxlen=max_len)

a sample speech

In [26]:
print(train.speech[2])
print(train_sequences[2])

δω ανάλογη ευαισθησία επιθέσει βάρος μελής κόμματος δέχομαι επιθέσει γραφείο επανέλθω επικοινωνία φοιτητής δήλωσαν στοιχείο κουκουλοφόρου διάφορους λυμαίνομαι χώρου προβαίνω καταγγελία ζητείται συνεχής επέμβαση αστυνομία πανεπιστημίο εκ φοιτητής χώρος άσυλο ιδεή αριστερός αριστερός ιδεολογίε εκφράζομαι πανεπιστήμιο θέλω εγγυηθώ πολιτεία ασφάλεί θέλω κάνω εκδήλωση παράδειγμα σχετικά εθνικά θέμαα ιδεολογίας συνάδω αριστερός ιδεολογίας βλέπω συλλήψει επέμβαση ελας κατάθεση ερώτησή νοεμβρίου τυχαίο βλέπω ανάλογος καταλογισμό ευθύνη πρυτανικός αρχές λέω κουκουλοφόρου τονίζω χρόνια ομάδα δρω δρω οργανωμένες συμμορία επιθέσει αστυνομικός πολιτής φοιτητής φυσικά χρησιμοποιώντας είδους όπλο εκρηκτικά απαραίτητες κουκούλα ειδικό πάγκο πανεπιστήμιο δίνω δώρο χρυσός αυγή βγάλω κουκούλες πρυτανικός αρχές εγκληματία λογοδοτήσω σύντομα δικαιοσύνη προβλέπομαι παράγραφο ποινικός κώδικας τονίζω εκ προτέρος αποφάσει συνήθως επισημαίνω ανομία ατιμωρησία έλυσαν προβλήματο αντιθέτως απρόβλεπτες συνέπεια δημ

this is a reverse function/ given the sequence returns the actual speech

In [27]:
reverse_word_index = dict([(value,key) for (key,value)in word_index.items()])

In [28]:
def decode(text):
  return " ".join([reverse_word_index.get(i,"?") for i in text])

In [29]:
decode(train_sequences[2])

'δω ανάλογη ευαισθησία επιθέσει βάρος μελής κόμματος δέχομαι επιθέσει γραφείο επανέλθω επικοινωνία φοιτητής δήλωσαν στοιχείο κουκουλοφόρου διάφορους λυμαίνομαι χώρου προβαίνω καταγγελία ζητείται συνεχής επέμβαση αστυνομία πανεπιστημίο εκ φοιτητής χώρος άσυλο ιδεή αριστερός αριστερός ιδεολογίε εκφράζομαι πανεπιστήμιο θέλω εγγυηθώ πολιτεία ασφάλεί θέλω κάνω εκδήλωση παράδειγμα σχετικά εθνικά θέμαα ιδεολογίας συνάδω αριστερός ιδεολογίας βλέπω συλλήψει επέμβαση ελας κατάθεση ερώτησή νοεμβρίου τυχαίο βλέπω ανάλογος καταλογισμό ευθύνη πρυτανικός αρχές λέω κουκουλοφόρου τονίζω χρόνια ομάδα δρω δρω οργανωμένες συμμορία επιθέσει αστυνομικός πολιτής φοιτητής φυσικά χρησιμοποιώντας είδους όπλο εκρηκτικά απαραίτητες κουκούλα ειδικό πάγκο πανεπιστήμιο δίνω δώρο χρυσός αυγή βγάλω κουκούλες πρυτανικός αρχές εγκληματία λογοδοτήσω σύντομα δικαιοσύνη προβλέπομαι παράγραφο ποινικός κώδικας τονίζω εκ προτέρος αποφάσει συνήθως επισημαίνω ανομία ατιμωρησία έλυσαν προβλήματο αντιθέτως απρόβλεπτες συνέπεια δη

yet again the shapes of the 3 different padded DF's

In [30]:
print(f"Shape of train {train_padded.shape}")
print(f"Shape of val {val_padded.shape}")
print(f"Shape of test {test_padded.shape}")

Shape of train (306000, 150)
Shape of val (54000, 150)
Shape of test (90000, 150)


This is my first model 

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_words, 
                              64,
                              input_length=max_len,
        # Use masking to handle the variable sequence lengths.
        mask_zero=True),
    tf.keras.layers.LSTM(124), # 64 is the dimension of the output.
    tf.keras.layers.Dense(124, activation='relu'),
    tf.keras.layers.Dense(15,activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           2082496   
_________________________________________________________________
lstm (LSTM)                  (None, 124)               93744     
_________________________________________________________________
dense (Dense)                (None, 124)               15500     
_________________________________________________________________
dense_1 (Dense)              (None, 15)                1875      
Total params: 2,193,615
Trainable params: 2,193,615
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=tf.keras.metrics.CategoricalAccuracy())

one-hot the Y's in order to use CategoricalCrossentropy

In [33]:
train_labels = tf.one_hot(train_labels,depth=15)
val_labels = tf.one_hot(val_labels,depth=15)
test_labels = tf.one_hot(test_labels,depth=15)

In [34]:
test_labels[4]

<tf.Tensor: shape=(15,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
      dtype=float32)>

below is the best model of all i tested. I tried different batch sizes(32,64,100,150), different max_len (150,300) and different outputs of LSTM (32,64,124). I also added a sigmoid activation in the last layer. 






In [35]:
#model 1 with embedding len 64
epochs = 10
history = model.fit(
   train_padded,
   train_labels,
    validation_data=(val_padded,val_labels),
    batch_size=64,
    shuffle=True,
    epochs=epochs)

Epoch 1/10
4782/4782 [==============================] - 183s 36ms/step - loss: 1.8844 - categorical_accuracy: 0.3731 - val_loss: 1.0113 - val_categorical_accuracy: 0.6627
Epoch 2/10
4782/4782 [==============================] - 170s 36ms/step - loss: 0.8650 - categorical_accuracy: 0.7059 - val_loss: 0.8108 - val_categorical_accuracy: 0.7250
Epoch 3/10
4782/4782 [==============================] - 169s 35ms/step - loss: 0.6401 - categorical_accuracy: 0.7772 - val_loss: 0.7166 - val_categorical_accuracy: 0.7596
Epoch 4/10
4782/4782 [==============================] - 168s 35ms/step - loss: 0.5026 - categorical_accuracy: 0.8226 - val_loss: 0.6910 - val_categorical_accuracy: 0.7722
Epoch 5/10
4782/4782 [==============================] - 168s 35ms/step - loss: 0.4062 - categorical_accuracy: 0.8576 - val_loss: 0.6872 - val_categorical_accuracy: 0.7870
Epoch 6/10
4782/4782 [==============================] - 168s 35ms/step - loss: 0.3345 - categorical_accuracy: 0.8834 - val_loss: 0.7503 - val_cat

loss and accuracy metrics 

In [36]:
loss, accuracy = model.evaluate(x=test_padded,y=test_labels)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

2813/2813 [==============================] - 14s 5ms/step - loss: 1.0173 - categorical_accuracy: 0.7982
Loss:  1.0172538757324219
Accuracy:  0.7982444167137146


that's the end of part 3 - Neural Network